In [36]:
import torchvision.datasets as datasets
# mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=None)

In [37]:
import torch as th
import syft as sy

hook = sy.TorchHook(th)

In [38]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
trusted_agg = sy.VirtualWorker(hook, id = "trusted_agg")

bob.clear_objects()
alice.clear_objects()
trusted_agg.clear_objects()

<VirtualWorker id:trusted_agg #objects:0>

In [39]:
device = th.device("cuda")

In [40]:
import torchvision.transforms as transforms

federated_train_loader = sy.FederatedDataLoader(
    datasets.MNIST(root='./data', train=True, download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])).federate((bob, alice)), 
                                                batch_size=64, shuffle=True)

test_loader = th.utils.data.DataLoader(
    datasets.MNIST('./data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=64, shuffle=True,)

In [41]:
# bob_train_dataset = sy.BaseDataset(train_data[:train_idx], train_labels[:train_idx]).send(bob)
# bob_test_dataset = sy.BaseDataset(test_data[:test_idx], test_labels[:test_idx]).send(bob)

# alice_train_dataset = sy.BaseDataset(train_data[train_idx:], train_labels[train_idx:]).send(alice)
# alice_test_dataset = sy.BaseDataset(test_data[test_idx:], test_labels[test_idx:]).send(alice)


In [42]:
# federated_train_dataset = sy.FederatedDataset([bob_train_dataset, alice_train_dataset])
# federated_test_dataset = sy.FederatedDataset([bob_test_dataset, alice_test_dataset])

# federated_train_dataloader = sy.FederatedDataLoader(federated_train_dataset, shuffle=True, batch_size = 64)
# federated_test_dataloader = sy.FederatedDataLoader(federated_test_dataset, shuffle=True, batch_size = 64)

In [43]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim = 1)

In [44]:
model = Net().to(device)

In [45]:
def train(model, federate_train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(federate_train_loader):
        model.send(data.location)
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        model.get()
        if batch_idx % 10 == 0:
            loss = loss.get()
            print(f'Train Epoch: {epoch} [{batch_idx*64}/{len(federate_train_loader)*64} ({100. * batch_idx / len(federated_train_loader)}%)] \t Loss: {loss.item()}')

In [48]:
def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with th.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction= 'sum').item() #sum up batch loss
            pred = output.argmax(1, keepdim = True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.01) 
epoch = 20
for epoch in range(1, epoch + 1):
    train(model, federated_train_loader, optimizer, epoch)
    test(model, test_loader)

th.save(model.state_dict(), "mnist_cnn.pt")

Train Epoch: 1 [0/60032 (0.0%)] 	 Loss: 0.5493183732032776
Train Epoch: 1 [640/60032 (1.0660980810234542%)] 	 Loss: 0.6795970797538757
Train Epoch: 1 [1280/60032 (2.1321961620469083%)] 	 Loss: 0.856945276260376
Train Epoch: 1 [1920/60032 (3.1982942430703623%)] 	 Loss: 0.6018005013465881
Train Epoch: 1 [2560/60032 (4.264392324093817%)] 	 Loss: 0.5160875916481018
Train Epoch: 1 [3200/60032 (5.330490405117271%)] 	 Loss: 0.5883960723876953
Train Epoch: 1 [3840/60032 (6.3965884861407245%)] 	 Loss: 0.9034500122070312
Train Epoch: 1 [4480/60032 (7.462686567164179%)] 	 Loss: 0.7067348957061768
Train Epoch: 1 [5120/60032 (8.528784648187633%)] 	 Loss: 0.6554229259490967
Train Epoch: 1 [5760/60032 (9.594882729211088%)] 	 Loss: 0.4344828128814697
Train Epoch: 1 [6400/60032 (10.660980810234541%)] 	 Loss: 0.6617513298988342
Train Epoch: 1 [7040/60032 (11.727078891257996%)] 	 Loss: 0.5518676042556763
Train Epoch: 1 [7680/60032 (12.793176972281449%)] 	 Loss: 0.7054015398025513
Train Epoch: 1 [8320/600

Train Epoch: 2 [7680/60032 (12.793176972281449%)] 	 Loss: 0.6588570475578308
Train Epoch: 2 [8320/60032 (13.859275053304904%)] 	 Loss: 0.5268580317497253
Train Epoch: 2 [8960/60032 (14.925373134328359%)] 	 Loss: 0.24381917715072632
Train Epoch: 2 [9600/60032 (15.991471215351812%)] 	 Loss: 0.4789677858352661
Train Epoch: 2 [10240/60032 (17.057569296375267%)] 	 Loss: 0.42705070972442627
Train Epoch: 2 [10880/60032 (18.12366737739872%)] 	 Loss: 0.4165276288986206
Train Epoch: 2 [11520/60032 (19.189765458422176%)] 	 Loss: 0.7013831734657288
Train Epoch: 2 [12160/60032 (20.255863539445627%)] 	 Loss: 0.3256555199623108
Train Epoch: 2 [12800/60032 (21.321961620469082%)] 	 Loss: 0.29518306255340576
Train Epoch: 2 [13440/60032 (22.388059701492537%)] 	 Loss: 0.45505428314208984
Train Epoch: 2 [14080/60032 (23.454157782515992%)] 	 Loss: 0.6590161919593811
Train Epoch: 2 [14720/60032 (24.520255863539447%)] 	 Loss: 0.5553333163261414
Train Epoch: 2 [15360/60032 (25.586353944562898%)] 	 Loss: 0.3752

Train Epoch: 3 [15360/60032 (25.586353944562898%)] 	 Loss: 0.404274046421051
Train Epoch: 3 [16000/60032 (26.652452025586353%)] 	 Loss: 0.4639897048473358
Train Epoch: 3 [16640/60032 (27.718550106609808%)] 	 Loss: 0.8322294354438782
Train Epoch: 3 [17280/60032 (28.784648187633262%)] 	 Loss: 0.2509915232658386
Train Epoch: 3 [17920/60032 (29.850746268656717%)] 	 Loss: 0.2150626927614212
Train Epoch: 3 [18560/60032 (30.916844349680172%)] 	 Loss: 0.23653534054756165
Train Epoch: 3 [19200/60032 (31.982942430703623%)] 	 Loss: 0.3675850033760071
Train Epoch: 3 [19840/60032 (33.04904051172708%)] 	 Loss: 0.31141310930252075
Train Epoch: 3 [20480/60032 (34.11513859275053%)] 	 Loss: 0.21202297508716583
Train Epoch: 3 [21120/60032 (35.18123667377399%)] 	 Loss: 0.18090030550956726
Train Epoch: 3 [21760/60032 (36.24733475479744%)] 	 Loss: 0.3136182129383087
Train Epoch: 3 [22400/60032 (37.3134328358209%)] 	 Loss: 0.4116103947162628
Train Epoch: 3 [23040/60032 (38.37953091684435%)] 	 Loss: 0.5011206

Train Epoch: 4 [22400/60032 (37.3134328358209%)] 	 Loss: 0.3494512140750885
Train Epoch: 4 [23040/60032 (38.37953091684435%)] 	 Loss: 0.3136310577392578
Train Epoch: 4 [23680/60032 (39.44562899786781%)] 	 Loss: 0.5544831156730652
Train Epoch: 4 [24320/60032 (40.511727078891255%)] 	 Loss: 0.35231155157089233
Train Epoch: 4 [24960/60032 (41.57782515991471%)] 	 Loss: 0.28206682205200195
Train Epoch: 4 [25600/60032 (42.643923240938165%)] 	 Loss: 0.20724211633205414
Train Epoch: 4 [26240/60032 (43.71002132196162%)] 	 Loss: 0.2710801362991333
Train Epoch: 4 [26880/60032 (44.776119402985074%)] 	 Loss: 0.42047184705734253
Train Epoch: 4 [27520/60032 (45.84221748400853%)] 	 Loss: 0.2985580563545227
Train Epoch: 4 [28160/60032 (46.908315565031984%)] 	 Loss: 0.18855027854442596
Train Epoch: 4 [28800/60032 (47.97441364605544%)] 	 Loss: 0.23264040052890778
Train Epoch: 4 [29440/60032 (49.04051172707889%)] 	 Loss: 0.2870449125766754
Train Epoch: 4 [30080/60032 (50.10660980810235%)] 	 Loss: 0.3445824

Train Epoch: 5 [29440/60032 (49.04051172707889%)] 	 Loss: 0.2874738276004791
Train Epoch: 5 [30080/60032 (50.10660980810235%)] 	 Loss: 0.15101918578147888
Train Epoch: 5 [30720/60032 (51.172707889125796%)] 	 Loss: 0.33131757378578186
Train Epoch: 5 [31360/60032 (52.23880597014925%)] 	 Loss: 0.17101800441741943
Train Epoch: 5 [32000/60032 (53.304904051172706%)] 	 Loss: 0.4122413694858551
Train Epoch: 5 [32640/60032 (54.37100213219616%)] 	 Loss: 0.18504329025745392
Train Epoch: 5 [33280/60032 (55.437100213219615%)] 	 Loss: 0.2821393609046936
Train Epoch: 5 [33920/60032 (56.50319829424307%)] 	 Loss: 0.35722115635871887
Train Epoch: 5 [34560/60032 (57.569296375266525%)] 	 Loss: 0.14648589491844177
Train Epoch: 5 [35200/60032 (58.63539445628998%)] 	 Loss: 0.13702884316444397
Train Epoch: 5 [35840/60032 (59.701492537313435%)] 	 Loss: 0.2217128574848175
Train Epoch: 5 [36480/60032 (60.76759061833689%)] 	 Loss: 0.20891991257667542
Train Epoch: 5 [37120/60032 (61.833688699360344%)] 	 Loss: 0.32

...

Train Epoch: 11 [3200/60032 (5.330490405117271%)] 	 Loss: 0.26779139041900635
Train Epoch: 11 [3840/60032 (6.3965884861407245%)] 	 Loss: 0.20952171087265015
Train Epoch: 11 [4480/60032 (7.462686567164179%)] 	 Loss: 0.1294107884168625
Train Epoch: 11 [5120/60032 (8.528784648187633%)] 	 Loss: 0.3069232404232025
Train Epoch: 11 [5760/60032 (9.594882729211088%)] 	 Loss: 0.22056622803211212
Train Epoch: 11 [6400/60032 (10.660980810234541%)] 	 Loss: 0.23525911569595337
Train Epoch: 11 [7040/60032 (11.727078891257996%)] 	 Loss: 0.09868206083774567
Train Epoch: 11 [7680/60032 (12.793176972281449%)] 	 Loss: 0.12767678499221802
Train Epoch: 11 [8320/60032 (13.859275053304904%)] 	 Loss: 0.18867692351341248
Train Epoch: 11 [8960/60032 (14.925373134328359%)] 	 Loss: 0.2724462151527405
Train Epoch: 11 [9600/60032 (15.991471215351812%)] 	 Loss: 0.1808701604604721
Train Epoch: 11 [10240/60032 (17.057569296375267%)] 	 Loss: 0.25736477971076965
Train Epoch: 11 [10880/60032 (18.12366737739872%)] 	 Loss: 

Train Epoch: 12 [9600/60032 (15.991471215351812%)] 	 Loss: 0.1424078345298767
Train Epoch: 12 [10240/60032 (17.057569296375267%)] 	 Loss: 0.23251232504844666
Train Epoch: 12 [10880/60032 (18.12366737739872%)] 	 Loss: 0.2702755928039551
Train Epoch: 12 [11520/60032 (19.189765458422176%)] 	 Loss: 0.16873987019062042
Train Epoch: 12 [12160/60032 (20.255863539445627%)] 	 Loss: 0.30403560400009155
Train Epoch: 12 [12800/60032 (21.321961620469082%)] 	 Loss: 0.12238368391990662
Train Epoch: 12 [13440/60032 (22.388059701492537%)] 	 Loss: 0.14385825395584106
Train Epoch: 12 [14080/60032 (23.454157782515992%)] 	 Loss: 0.1591998040676117
Train Epoch: 12 [14720/60032 (24.520255863539447%)] 	 Loss: 0.15363356471061707
Train Epoch: 12 [15360/60032 (25.586353944562898%)] 	 Loss: 0.12513145804405212
Train Epoch: 12 [16000/60032 (26.652452025586353%)] 	 Loss: 0.1519988477230072
Train Epoch: 12 [16640/60032 (27.718550106609808%)] 	 Loss: 0.1963045746088028
Train Epoch: 12 [17280/60032 (28.78464818763326

Train Epoch: 13 [16000/60032 (26.652452025586353%)] 	 Loss: 0.044055238366127014
Train Epoch: 13 [16640/60032 (27.718550106609808%)] 	 Loss: 0.17087450623512268
Train Epoch: 13 [17280/60032 (28.784648187633262%)] 	 Loss: 0.059706442058086395
Train Epoch: 13 [17920/60032 (29.850746268656717%)] 	 Loss: 0.2748503088951111
Train Epoch: 13 [18560/60032 (30.916844349680172%)] 	 Loss: 0.25084438920021057
Train Epoch: 13 [19200/60032 (31.982942430703623%)] 	 Loss: 0.14452679455280304
Train Epoch: 13 [19840/60032 (33.04904051172708%)] 	 Loss: 0.2365816831588745
Train Epoch: 13 [20480/60032 (34.11513859275053%)] 	 Loss: 0.14566156268119812
Train Epoch: 13 [21120/60032 (35.18123667377399%)] 	 Loss: 0.3126400113105774
Train Epoch: 13 [21760/60032 (36.24733475479744%)] 	 Loss: 0.24386601150035858
Train Epoch: 13 [22400/60032 (37.3134328358209%)] 	 Loss: 0.12942826747894287
Train Epoch: 13 [23040/60032 (38.37953091684435%)] 	 Loss: 0.13132713735103607
Train Epoch: 13 [23680/60032 (39.44562899786781%

Train Epoch: 14 [22400/60032 (37.3134328358209%)] 	 Loss: 0.26366478204727173
Train Epoch: 14 [23040/60032 (38.37953091684435%)] 	 Loss: 0.1678014099597931
Train Epoch: 14 [23680/60032 (39.44562899786781%)] 	 Loss: 0.10457220673561096
Train Epoch: 14 [24320/60032 (40.511727078891255%)] 	 Loss: 0.05282852053642273
Train Epoch: 14 [24960/60032 (41.57782515991471%)] 	 Loss: 0.37414979934692383
Train Epoch: 14 [25600/60032 (42.643923240938165%)] 	 Loss: 0.12071076780557632
Train Epoch: 14 [26240/60032 (43.71002132196162%)] 	 Loss: 0.06461441516876221
Train Epoch: 14 [26880/60032 (44.776119402985074%)] 	 Loss: 0.17860327661037445
Train Epoch: 14 [27520/60032 (45.84221748400853%)] 	 Loss: 0.33421462774276733
Train Epoch: 14 [28160/60032 (46.908315565031984%)] 	 Loss: 0.16937585175037384
Train Epoch: 14 [28800/60032 (47.97441364605544%)] 	 Loss: 0.1719464808702469
Train Epoch: 14 [29440/60032 (49.04051172707889%)] 	 Loss: 0.09912234544754028
Train Epoch: 14 [30080/60032 (50.10660980810235%)] 

Train Epoch: 15 [28800/60032 (47.97441364605544%)] 	 Loss: 0.2080739587545395
Train Epoch: 15 [29440/60032 (49.04051172707889%)] 	 Loss: 0.16449688374996185
Train Epoch: 15 [30080/60032 (50.10660980810235%)] 	 Loss: 0.06201435998082161
Train Epoch: 15 [30720/60032 (51.172707889125796%)] 	 Loss: 0.11111438274383545
Train Epoch: 15 [31360/60032 (52.23880597014925%)] 	 Loss: 0.21094626188278198
Train Epoch: 15 [32000/60032 (53.304904051172706%)] 	 Loss: 0.20333929359912872
Train Epoch: 15 [32640/60032 (54.37100213219616%)] 	 Loss: 0.0810951292514801
Train Epoch: 15 [33280/60032 (55.437100213219615%)] 	 Loss: 0.289828896522522
Train Epoch: 15 [33920/60032 (56.50319829424307%)] 	 Loss: 0.11391934752464294
Train Epoch: 15 [34560/60032 (57.569296375266525%)] 	 Loss: 0.2071499526500702
Train Epoch: 15 [35200/60032 (58.63539445628998%)] 	 Loss: 0.07873706519603729
Train Epoch: 15 [35840/60032 (59.701492537313435%)] 	 Loss: 0.13353823125362396
Train Epoch: 15 [36480/60032 (60.76759061833689%)] 	

In [ ]:
import matplotlib.pyplot as plt

image = mnist_trainset.data[52009]
img = image.view(1, 784).float()
with th.no_grad():
    logps = model(img)

ps = th.exp(logps)
probab = list(ps.numpy())
print("Predicted Digit =", probab.index(max(probab)))
plt.imshow(img.view(28, 28))

In [ ]:
sy.BaseDataset()